<a href="https://colab.research.google.com/github/armandossrecife/my_validation/blob/main/avalia_modelo_inspecao2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa os dados para avaliação

In [73]:
!wget https://raw.githubusercontent.com/armandossrecife/my_validation/main/cassandra_issues_inspected_merged.xlsx

--2023-11-20 21:58:00--  https://raw.githubusercontent.com/armandossrecife/my_validation/main/cassandra_issues_inspected_merged.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 576488 (563K) [application/octet-stream]
Saving to: ‘cassandra_issues_inspected_merged.xlsx’

cassandra_issues_in 100%[===================>] 562.98K  --.-KB/s    in 0.05s   

2023-11-20 21:58:01 (11.4 MB/s) - ‘cassandra_issues_inspected_merged.xlsx’ saved [576488/576488]



# Funções de apoio


In [74]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats as stats
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def convert_to_upper(list_of_strings):
  upper_list = []
  for string in list_of_strings:
    upper_list.append(string.upper())
  return upper_list

def conta_yes_no(my_list):
  conta_yes, conta_no = 0,0
  for each in my_list:
    if each == 'YES':
      conta_yes += 1
    else:
      conta_no += 1
  return conta_yes, conta_no

def convert_yes_no_to_int(dfa, dfb, my_column):
  labels = ["YES", "NO"]  # Define your possible labels here

  # Convert labels to numerical values
  label_to_num = {label: i for i, label in enumerate(labels)}
  dfa[my_column] = dfa[my_column].map(label_to_num)
  dfb[my_column] = dfb[my_column].map(label_to_num)
  return dfa, dfb

# Define a function to calculate Cohen's Kappa
def calculate_cohens_kappa(dfa, dfb, my_column):
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(dfa[my_column], dfb[my_column])
    return kappa

def extra_metrics_yes(dfa, dfb, my_column):
  # Define your ground truth (actual values) and predicted values
  ground_truth = dfa[my_column]  # Ground truth values from manual inspection
  predicted = dfb[my_column]     # Predicted values from ChatGPT inspection

  # Calculate precision, recall, accuracy, and F1-score
  precision = precision_score(ground_truth, predicted, pos_label=1)
  recall = recall_score(ground_truth, predicted, pos_label=1)
  accuracy = accuracy_score(ground_truth, predicted)
  f1 = f1_score(ground_truth, predicted, pos_label=1)
  return precision, recall, accuracy, f1

# Importa dados originais

Merge da inspeção manual e inspeção via ChatGPt

In [75]:
df_inspection = pd.read_excel('cassandra_issues_inspected_merged.xlsx')
df_inspection

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual,architectural_impact_ai,TP,TN,FP,FN
0,CASSANDRA-173,add getPendingTasks to CFSMBean,Improvement,Resolved,Low,need to add an atomicint and inc/decr it whene...,['rebased patch as 0001-CASSANDRA-173-added-CF...,NO,NO,0,1,0,0
1,CASSANDRA-124,NullPointerException in consistency manager af...,Bug,Resolved,Urgent,ERROR [CONSISTENCY-MANAGER:2] 2009-04-30 18:22...,"[""Shouldn't ConsistencyManager() constructor c...",NO,YES,0,0,1,0
2,CASSANDRA-78,Interrupted recovery requires manual intervent...,Bug,Resolved,Urgent,"Originally reported by Alexander Staubo: ""If y...","[""Prashant's memory seems to be wrong here -- ...",NO,YES,0,0,1,0
3,CASSANDRA-34,Hinted handoff rows never get deleted,Bug,Resolved,Normal,"from the list: ""after the hints are delivered,...",['Sending out hinted data needs correct RowMut...,NO,YES,0,0,1,0
4,CASSANDRA-447,Ability to temporary set minimum and maximum c...,New Feature,Resolved,Normal,We need the ability to temporary set minimum a...,['Added the ability set/get max/min compaction...,NO,YES,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,CASSANDRA-18430,When decommissioning should set Severity to li...,Improvement,Resolved,Normal,When we are decommissioning we first set LEAVI...,['Can you rebase this to more relevant commits...,YES,NO,0,0,0,1
222,CASSANDRA-18320,Incompatible file system thrown while running ...,Bug,Resolved,Normal,{code}\n\njava.io.UncheckedIOException\n\n\tat...,['It says that it can not recursively remove a...,YES,NO,0,0,0,1
223,CASSANDRA-18153,Memtable being flushed without hostId in versi...,Bug,Resolved,Normal,On ticket CASSANDRA-16619 some files were chan...,"['CC [~jlewandowski]\xa0', 'Storage service is...",YES,NO,0,0,0,1
224,CASSANDRA-18681,Internode legacy SSL storage port certificate ...,Bug,Resolved,Normal,In CASSANDRA-16666 the SSLContext cache was ch...,['4.1 [Branch|https://github.com/jonmeredith/c...,YES,NO,0,0,0,1


In [76]:
df_inspection_check = df_inspection[['issue_key', 'issue_type', 'summary', 'architectural_impact_manual','architectural_impact_ai', 'TP', 'TN', 'FP', 'FN']]
df_inspection_check

,issue_key,issue_type,summary,architectural_impact_manual,architectural_impact_ai,TP,TN,FP,FN
0,CASSANDRA-173,Improvement,add getPendingTasks to CFSMBean,NO,NO,0,1,0,0
1,CASSANDRA-124,Bug,NullPointerException in consistency manager af...,NO,YES,0,0,1,0
2,CASSANDRA-78,Bug,Interrupted recovery requires manual intervent...,NO,YES,0,0,1,0
3,CASSANDRA-34,Bug,Hinted handoff rows never get deleted,NO,YES,0,0,1,0
4,CASSANDRA-447,New Feature,Ability to temporary set minimum and maximum c...,NO,YES,0,0,1,0
...,...,...,...,...,...,...,...,...,...
221,CASSANDRA-18430,Improvement,When decommissioning should set Severity to li...,YES,NO,0,0,0,1
222,CASSANDRA-18320,Bug,Incompatible file system thrown while running ...,YES,NO,0,0,0,1
223,CASSANDRA-18153,Bug,Memtable being flushed without hostId in versi...,YES,NO,0,0,0,1
224,CASSANDRA-18681,Bug,Internode legacy SSL storage port certificate ...,YES,NO,0,0,0,1


In [77]:
true_positive = sum(df_inspection_check.TP)
true_negative = sum(df_inspection_check.TN)
false_positive = sum(df_inspection_check.FP)
false_negative = sum(df_inspection_check.FN)

print(f'Soma das métricas: {true_positive+true_negative+false_positive+false_negative}')
print(f'True Positive: {true_positive}, True Negative: {true_negative}, False Positive: {false_positive}, False Negative: {false_negative}')

Soma das métricas: 226
True Positive: 90, True Negative: 118, False Positive: 6, False Negative: 12


# Dados da inspeção manual

In [78]:
df_inspection_check_manual = df_inspection_check[['issue_key', 'issue_type','summary','architectural_impact_manual']]
df_inspection_check_manual['architectural_impact'] = df_inspection_check_manual['architectural_impact_manual'].str.upper()
df_inspection_check_manual

<ipython-input-78-9024a91141ee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_manual['architectural_impact'] = df_inspection_check_manual['architectural_impact_manual'].str.upper()


,issue_key,issue_type,summary,architectural_impact_manual,architectural_impact
0,CASSANDRA-173,Improvement,add getPendingTasks to CFSMBean,NO,NO
1,CASSANDRA-124,Bug,NullPointerException in consistency manager af...,NO,NO
2,CASSANDRA-78,Bug,Interrupted recovery requires manual intervent...,NO,NO
3,CASSANDRA-34,Bug,Hinted handoff rows never get deleted,NO,NO
4,CASSANDRA-447,New Feature,Ability to temporary set minimum and maximum c...,NO,NO
...,...,...,...,...,...
221,CASSANDRA-18430,Improvement,When decommissioning should set Severity to li...,YES,YES
222,CASSANDRA-18320,Bug,Incompatible file system thrown while running ...,YES,YES
223,CASSANDRA-18153,Bug,Memtable being flushed without hostId in versi...,YES,YES
224,CASSANDRA-18681,Bug,Internode legacy SSL storage port certificate ...,YES,YES


In [79]:
ai_manual = df_inspection_check_manual.architectural_impact.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_manual)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 226, Yes: 102, No: 124
%Yes: 45.13, %No: 54.87


# Dados da inspeção via ChatGPT

In [80]:
df_inspection_check_chagpt = df_inspection_check[['issue_key', 'issue_type','summary','architectural_impact_ai']]
df_inspection_check_chagpt['architectural_impact'] = df_inspection_check_chagpt['architectural_impact_ai']
df_inspection_check_chagpt['architectural_impact'] = df_inspection_check_chagpt['architectural_impact_ai'].str.upper()
df_inspection_check_chagpt = df_inspection_check_chagpt[['issue_key', 'issue_type', 'summary', 'architectural_impact']]
df_inspection_check_chagpt

<ipython-input-80-b96b823007ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['architectural_impact'] = df_inspection_check_chagpt['architectural_impact_ai']
<ipython-input-80-b96b823007ac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['architectural_impact'] = df_inspection_check_chagpt['architectural_impact_ai'].str.upper()


,issue_key,issue_type,summary,architectural_impact
0,CASSANDRA-173,Improvement,add getPendingTasks to CFSMBean,NO
1,CASSANDRA-124,Bug,NullPointerException in consistency manager af...,YES
2,CASSANDRA-78,Bug,Interrupted recovery requires manual intervent...,YES
3,CASSANDRA-34,Bug,Hinted handoff rows never get deleted,YES
4,CASSANDRA-447,New Feature,Ability to temporary set minimum and maximum c...,YES
...,...,...,...,...
221,CASSANDRA-18430,Improvement,When decommissioning should set Severity to li...,NO
222,CASSANDRA-18320,Bug,Incompatible file system thrown while running ...,NO
223,CASSANDRA-18153,Bug,Memtable being flushed without hostId in versi...,NO
224,CASSANDRA-18681,Bug,Internode legacy SSL storage port certificate ...,NO


In [81]:
ai_chatgpt = df_inspection_check_chagpt.architectural_impact.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_chatgpt)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 226, Yes: 96, No: 130
%Yes: 42.48, %No: 57.52


# Cálculo do Kappa


In [82]:
# Converte a coluna architecture_issue para 0 (NO) ou 1 (YES)
dfa, dfb = convert_yes_no_to_int(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architectural_impact')

<ipython-input-74-fb1b080fa0ff>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa[my_column] = dfa[my_column].map(label_to_num)


In [83]:
kappa_manual_chatgpt = calculate_cohens_kappa(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architectural_impact')
print(round(kappa_manual_chatgpt, 3))

0.838


# Cálculo da performance do modelo de inspeção ChatGPT

In [84]:
my_precision, my_recall, my_accuracy, my_f1_score = extra_metrics_yes(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architectural_impact')
print(f'my_precision: {round(my_precision, 3)}, my_recall: {round(my_recall, 3)}, my_accuracy: {round(my_accuracy, 3)}, my_f1_score: {round(my_f1_score,3)}')

my_precision: 0.908, my_recall: 0.952, my_accuracy: 0.92, my_f1_score: 0.929
